<a href="https://colab.research.google.com/github/mshahmeer1/Radiological-phenotypes-of-ARDS/blob/main/Embedding_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run this cell if running notebook from Colab
!git clone https://github.com/Google-Health/imaging-research.git
!mv imaging-research/cxr-foundation/cxr_foundation .


Cloning into 'imaging-research'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 301 (delta 15), reused 3 (delta 3), pack-reused 274
Receiving objects: 100% (301/301), 292.36 KiB | 14.62 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [ ]:
!pip install google-cloud-storage==1.42.3 \
    pandas==1.3.5 \
    tensorflow==2.10.0 \
    tf-models-official==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 KB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import auth

# Authenticate user for access. There will be a popup asking you to sign in with your user and approve access.
auth.authenticate_user()

In [ ]:
import os

from google.cloud import storage
import pandas as pd

def download_blob(bucket, source_blob_name: str, destination_file_name: str):
    """
    Downloads a blob from the bucket.
    
    https://cloud.google.com/storage/docs/downloading-objects
    """
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Downloaded {source_blob_name}")


storage_client = storage.Client()

# Make a directory to download the data
if not os.path.exists('data'):
  os.mkdir('data')

In [ ]:
embeddings_bucket = storage_client.bucket(
    'image-embeddings-mimic-cxr-1.0.physionet.org')    

# Download the checksums file which contains a records list
download_blob(embeddings_bucket, "SHA256SUMS.txt", "data/SHA256SUMS.txt")

Downloaded SHA256SUMS.txt


In [ ]:
df_embeddings = pd.read_csv("data/SHA256SUMS.txt", delimiter=" ", header=None, skiprows=[0])  # Skip the license file entry
display(df_embeddings.head())

,0,1
0,70e8c87d38d5dfba586218ccafb7428f76a3c7b08f6179...,files/p10/p10000032/s50414267/02aa804e-bde0afd...
1,a0c7a01d9fc223b25767ebf04d6f736ac4cca1f9336946...,files/p10/p10000032/s53189527/2a2277a9-b0ded15...
2,38202feabe98d8cd042b50879a886c8a099aba9afef18c...,files/p10/p10000032/s53911762/68b5c4b1-227d048...
3,23283d7a2cb72d8e59f21dfd33d66be9082f69b18c5c81...,files/p10/p10000032/s53911762/fffabebf-74fd3a1...
4,c62e51c914c91b2bc2d3c14a82046709b800bada664e5c...,files/p10/p10000032/s56699142/ea030e7a-2e3b134...


In [ ]:
import re

# Example: 'files/p19/p19692222/s59566639/965b6053-a2c70d67-c0467ca6-02372346-fb7c6224.tfrecord'
FILE_PATTERN = re.compile(r"files/(?:\w+)/p(?P<subject_id>\w+)/s(?P<study_id>\w+)/(?P<dicom_id>[\w-]+)\.tfrecord")

def parse_file_pattern(file_path: str):
    """
    Extracts the subject_id, study_id, and dicom_id
    from the full file path string.
    """
    match = FILE_PATTERN.fullmatch(file_path)
    if not match:
        raise Exception(f"Failed to match file path: {file_path}")
    return (int(match[1]), int(match[2]), match[3])

# Create additional columns from file path components
df_embeddings = df_embeddings[[1]]
df_embeddings.rename(columns={1: "embedding_file"}, inplace=True)
df_embeddings[["subject_id","study_id", "dicom_id"]] = df_embeddings.apply(
    lambda x: parse_file_pattern(x["embedding_file"]), axis=1, result_type="expand")

df_embeddings.head(10)

,embedding_file,subject_id,study_id,dicom_id
0,files/p10/p10000032/s50414267/02aa804e-bde0afd...,10000032,50414267,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014
1,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,10000032,53189527,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab
2,files/p10/p10000032/s53911762/68b5c4b1-227d048...,10000032,53911762,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714
3,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,10000032,53911762,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818
4,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,10000032,56699142,ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c
5,files/p10/p10000764/s57375967/096052b7-d256dc4...,10000764,57375967,096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4
6,files/p10/p10000898/s50771383/2a280266-c8bae12...,10000898,50771383,2a280266-c8bae121-54d75383-cac046f4-ca37aa16
7,files/p10/p10000898/s54205396/b75df1bd-0f22d63...,10000898,54205396,b75df1bd-0f22d631-52d73526-2ae7b85a-d843b39d
8,files/p10/p10000935/s50578979/d0b71acc-b5a6204...,10000935,50578979,d0b71acc-b5a62046-bbb5f6b8-7b173b85-65cdf738
9,files/p10/p10000935/s51178377/9b314ad7-fbcb042...,10000935,51178377,9b314ad7-fbcb0422-6db62dfc-732858d0-a5527d8b


In [ ]:
final_dicom = pd.read_csv('final_dicom.csv')
final_dicom = final_dicom.rename(columns={"dicom": "dicom_id"})
final_dicom = final_dicom.drop(['Unnamed: 0'], 
               axis=1)
print(final_dicom)

                                          dicom_id
0     7b25b3ed-e780a527-319cb7b3-02d5d071-f1cddee9
1     c1ad3e27-62d05ef8-95018fe3-b8bcfe4b-bbba0e1f
2     9b1a8a51-2b8e4a04-1719059d-aa6bc888-7ace612b
3     3813b9b6-88d998b4-941e767b-601ba7c1-98f61102
4     606ea60e-f3c5c58e-68fd84ca-db4e599b-127aa53e
...                                            ...
6332  b3de193f-980afff8-5f90661c-819d6823-6757f24d
6333  2eb70dfe-52fa728e-a36e09be-ec0ed3cf-0a2ea7f0
6334  ef49bac7-16939860-5a4f182e-c568720f-e0c9d278
6335  46510b80-411ac511-fe6ffab2-d7dfdc76-dff1a762
6336  f7e95a22-cb958055-47114ddf-38532ef4-b4c172d5

[6337 rows x 1 columns]


In [ ]:
Xray_embeddings = final_dicom.merge(df_embeddings, how='inner', on='dicom_id')

Xray_embeddings.head(10)

,dicom_id,embedding_file,subject_id,study_id
0,7b25b3ed-e780a527-319cb7b3-02d5d071-f1cddee9,files/p10/p10001884/s50712381/7b25b3ed-e780a52...,10001884,50712381
1,c1ad3e27-62d05ef8-95018fe3-b8bcfe4b-bbba0e1f,files/p10/p10001884/s56722923/c1ad3e27-62d05ef...,10001884,56722923
2,9b1a8a51-2b8e4a04-1719059d-aa6bc888-7ace612b,files/p10/p10001884/s59305618/9b1a8a51-2b8e4a0...,10001884,59305618
3,3813b9b6-88d998b4-941e767b-601ba7c1-98f61102,files/p10/p10004235/s52379321/3813b9b6-88d998b...,10004235,52379321
4,606ea60e-f3c5c58e-68fd84ca-db4e599b-127aa53e,files/p10/p10004235/s57318275/606ea60e-f3c5c58...,10004235,57318275
5,5b05c3da-4f8f9c06-7b8c4faf-4c12d978-6cb22b83,files/p10/p10004235/s52604377/5b05c3da-4f8f9c0...,10004235,52604377
6,c24939ff-cf96a7e2-dcc4a608-a9f63b02-2b64eca1,files/p10/p10004235/s58604118/c24939ff-cf96a7e...,10004235,58604118
7,888557ac-5bbba51e-dce5b075-8e281e2d-348e55bf,files/p10/p10011365/s51830718/888557ac-5bbba51...,10011365,51830718
8,b08b18a9-635a6e56-979cf9d6-314a48d8-0f833747,files/p10/p10011365/s54847293/b08b18a9-635a6e5...,10011365,54847293
9,afea1d4e-5402c28c-2aeb480f-c9d93494-7e04879d,files/p10/p10011365/s53869555/afea1d4e-5402c28...,10011365,53869555


In [ ]:
print(Xray_embeddings['embedding_file'][0])
print(Xray_embeddings['dicom_id'][0])

files/p10/p10001884/s50712381/7b25b3ed-e780a527-319cb7b3-02d5d071-f1cddee9.tfrecord
7b25b3ed-e780a527-319cb7b3-02d5d071-f1cddee9


In [ ]:
EMBEDDINGS_DIR = 'data/mimic-embeddings-files'

if not os.path.exists(EMBEDDINGS_DIR):
  os.mkdir(EMBEDDINGS_DIR)

In [ ]:
# Download embeddings
for i, row in Xray_embeddings.iterrows():
    download_blob(embeddings_bucket, row["embedding_file"], f"{EMBEDDINGS_DIR}/{row['dicom_id']}.tfrecord")
    


Streaming output truncated to the last 5000 lines.
Downloaded files/p11/p11962176/s54435424/6ac23357-3197b88d-eb1e8c53-8a73993c-75badf1d.tfrecord
Downloaded files/p11/p11962176/s51118536/516646f1-817a14ba-a04a4bfc-4b6206f4-331e747e.tfrecord
Downloaded files/p11/p11962176/s53583027/7242bdce-d75141b1-7896338a-7455317f-d2d44636.tfrecord
Downloaded files/p11/p11963546/s55810914/2d25a9b5-121989d4-23cac44e-2c3a7ea4-7eeee711.tfrecord
Downloaded files/p11/p11963546/s56469037/8de8c219-4b70f145-ec7f0dba-ba4e9bcc-be2d3146.tfrecord
Downloaded files/p11/p11963546/s54922105/ee954737-facfa2f4-584a1a99-63404d86-082d490b.tfrecord
Downloaded files/p11/p11963546/s55166970/3923809b-41a28d2e-4ea09552-2a6d7f37-84de547d.tfrecord
Downloaded files/p11/p11964706/s57253806/1f3c1c73-53164ef7-6fa88b7a-233a15b5-feb04f98.tfrecord
Downloaded files/p11/p11964706/s57376786/2458cc1a-e3c46fba-f9b43539-c12c6e4c-b42d2206.tfrecord
Downloaded files/p11/p11964706/s54476997/d522f347-dcb97f14-97fbe894-0088ca42-77219a86.tfrecord

In [ ]:
import glob

import tensorflow as tf

raw_dataset = tf.data.TFRecordDataset(glob.glob(f"{EMBEDDINGS_DIR}/*.tfrecord"))
for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

features {
  feature {
    key: "embedding"
    value {
      float_list {
        value: -0.8777239918708801
        value: -1.8964942693710327
        value: 0.4972558915615082
        value: -2.1113977432250977
        value: -0.35365352034568787
        value: 0.19328871369361877
        value: -0.22779887914657593
        value: 0.5724949240684509
        value: 1.1540465354919434
        value: 0.2173670530319214
        value: 0.1238325983285904
        value: -0.6285625100135803
        value: -1.4253352880477905
        value: 0.3985215723514557
        value: 2.0783064365386963
        value: 1.3014274835586548
        value: -0.5936800241470337
        value: 0.7240900993347168
        value: 0.6546916365623474
        value: 1.3955683708190918
        value: 1.0948511362075806
        value: 0.7062291502952576
        value: -2.795297622680664
        value: 1.4117610454559326
        value: 0.3257801830768585
        value: 1.5489912033081055
        value: -2.148270368576

In [ ]:
# EMBEDDINGS_DIR = 'data/mimic-embeddings-files'

# raw_dataset = tf.data.TFRecordDataset(glob.glob(f"{EMBEDDINGS_DIR}/*.tfrecord"))
# for raw_record in raw_dataset:
#   for Xray_embeddings['embedding_file'] in Xray_embeddings:
#     if raw_record.features.feature['image/id'][-78, -4].bytes_list.value == Xray_embeddings['embedding_file'][:,-9]
  
#     print(raw_record.features.feature['embedding'].float_list.value)
  

# files/p10/p10001884/s50712381/7b25b3ed-e780a527-319cb7b3-02d5d071-f1cddee9.tfrecord

# value: "gs://superrad/inputs/mimic-cxr/files/p11/p11900074/s54903837/c89cdd3a-6d1722c2-0ad41004-efd870dd-4250f9ad.dcm"

In [ ]:
# !pip install pandas-tfrecords

# import pandas as pd
# import pandas_tfrecords as pdtfr
# pdtfr.tfrecords_to_pandas(file_paths = r'/content/data/mimic-embeddings-files/01143881-ec37b41c-8e756f6d-253f65ac-107f080d.tfrecord')

In [ ]:
# def parse_df_element(element):
#     parser = {
        # 'embedding': tf.io.FixedLenFeature([], tf.float32),
        # 'image/format': tf.io.FixedLenFeature([], tf.string),
        # 'image/id': tf.io.FixedLenFeature([], tf.string),
#     }
#     # create an example:
#     content = tf.io.parse_single_example(element, parser)
#     return content['embedding'], \
#         content['image/format'], content['image/id']

In [ ]:
# parsed_tf_records = raw_dataset.map(parse_df_element)
# df = pd.DataFrame(
#     parsed_tf_records.as_numpy_iterator(),
#     columns=['embedding', 'image/format', 'image/id']
# )

In [ ]:
feature_description = {
        'embedding': tf.io.VarLenFeature(tf.float32),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/id': tf.io.FixedLenFeature([], tf.string), 
}
def parse_example(example_proto):
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)
  
    embedding = tf.sparse.to_dense(parsed_example['embedding'])
    image_format = parsed_example['image/format']
    image_id = parsed_example['image/id']
  
    return embedding, image_format, image_id


def embedding_to_df(path):
  
  tfrecord_file = path
  dataset = tf.data.TFRecordDataset(tfrecord_file)
  dataset = dataset.map(parse_example)

  data_list = []

  for embedding, image_format, image_id in dataset:
      data_list.append({
          'embedding': embedding.numpy(),
          'image/format': image_format.numpy().decode('utf-8'),
          'image/id': image_id.numpy().decode('utf-8')
      })

  df = pd.DataFrame(data_list)
  return df

In [ ]:

combined_df = pd.DataFrame(columns=['embedding', 'image/format', 'image/id'])

print(combined_df)


EMBEDDINGS_DIRECTORY = '/content/data/mimic-embeddings-files'
for i, row in Xray_embeddings.iterrows():

  tfrecord_file = f"{EMBEDDINGS_DIRECTORY}/{row['dicom_id']}.tfrecord"
  df = embedding_to_df(tfrecord_file)
  print(df.head())
  combined_df = pd.concat([combined_df,df], axis = 0)


pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)
print(combined_df)

Empty DataFrame
Columns: [embedding, image/format, image/id]
Index: []
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
Tensor("strided_slice:0", shape=(), dtype=int32)
(None,)
                                                                                                                                                                                                                                                     

In [ ]:
combined_df['Len'] = combined_df['embedding'].apply(len)

combined_df.head(50)

,embedding,image/format,image/id,Len
0,"[0.14432792, -1.0001783, 1.2237985, -2.5238967, 0.028393254, 0.12060432, 0.038533475, 0.14635453, 0.5236418, 0.057341967, 0.20065153, 0.20015016, -0.2075376, 0.3647379, 1.7438748, 1.4796498, -0.4825735, 0.8710406, 0.026959576, 1.6118201, 0.8285951, 0.3181009, -2.2628925, 1.1817429, -0.058578253, 1.1893989, -2.534076, -1.6747781, -0.19084167, 1.192593, -2.3331518, 0.29941976, 0.49515983, 0.5208489, -0.013177927, -0.81214553, -0.14054552, -0.5746485, -4.7142124, -2.8609195, -0.3336199, 0.085793436, -0.69387734, -0.05213939, 1.2485218, 0.06292598, 0.7448172, -2.6065915, 0.88135725, 0.64107245, -0.14517383, -1.3623407, 1.3315818, 0.4025075, 1.8144461, 0.7658751, 0.8764032, 1.4416577, -1.5358542, -0.72286636, 0.21001828, -2.231412, -0.0875946, -0.86823875, 0.64299226, -0.10463949, 1.9043921, 0.23621826, -0.85501206, -1.2456913, -1.734235, -0.96515596, 3.5325885, -1.4825721, 1.2580396, -0.11149041, 0.40217397, -0.7863048, 2.4016492, 1.9171268, 0.60018796, 0.39614904, -0.7111156, 1.8046997, -0.3448629, -4.277051, -1.3521482, 1.4122313, -2.2097154, 0.84989065, 0.115209125, -2.1821728, -0.6067679, -0.2914286, -1.5445822, -1.4812654, -1.4578083, 3.6084394, 0.70949244, 0.37720984, ...]",png,gs://superrad/inputs/mimic-cxr/files/p10/p10001884/s50712381/7b25b3ed-e780a527-319cb7b3-02d5d071-f1cddee9.dcm,1376
0,"[-0.7477663, -0.73449117, 1.1258107, -1.7856662, -0.70652115, -0.112519436, 0.40622452, 0.22852165, 1.1102792, 0.12069969, -0.1701393, 0.097142585, -1.1596818, 0.40880927, 1.8974357, 1.2107215, -0.6686534, 0.7276761, 0.95921236, 2.1944418, 1.2364907, 0.3085884, -2.7541192, 0.9947082, 1.0225627, 1.2418331, -1.9751889, -1.839491, -0.5239483, 1.0861287, -1.6300763, -0.005942755, -0.09047508, 0.46952015, -0.093820475, 0.031035902, -0.53611755, -0.18764584, -4.1279798, -2.6021478, -0.3427887, 0.22403072, 0.2880765, -0.67742467, 1.427065, 1.0741214, 0.7379849, -2.1338675, 1.1350923, 0.86686873, -0.60446674, -1.7012029, 1.4678123, 0.5057455, 1.8879908, 0.7587029, 0.8170383, 1.8316617, -0.9365972, 0.09277362, -0.9120427, -2.5226815, -0.23913205, -0.7035271, 0.50921774, -0.04192976, 1.271146, 0.08670681, -0.4727981, -1.5572284, -0.7471403, -0.18215089, 3.270215, -1.2052985, 1.5647906, -0.49289757, -0.108513705, -0.86401314, 1.5734912, 2.2302308, 1.4837343, -0.15498196, -0.79058856, 1.8182315, -0.9557299, -3.9753242, -1.152286, 0.93970275, -1.7772708, 0.7828179, 0.6948927, -2.2519252, -1.3077023, 0.37799767, -0.99352646, -0.8073111, -1.284207, 3.9095807, 0.08672662, 0.45669675, ...]",png,gs://superrad/inputs/mimic-cxr/files/p10/p10001884/s56722923/c1ad3e27-62d05ef8-95018fe3-b8bcfe4b-bbba0e1f.dcm,1376
0,"[-0.16570957, -1.1347905, 1.0465956, -1.3231037, -0.49246624, -0.16658545, 0.22532295, 0.59680384, 0.7918299, -0.1495825, 0.5386322, -0.19745253, -0.71077347, 0.38197267, 1.2766013, 1.0461882, -0.51092297, 0.37960693, 0.3001211, 1.3645428, 0.6283592, 0.31444576, -2.0026832, 0.5954331, 0.93290603, 0.79720795, -1.4304217, -1.1962162, -0.666557, 0.6791834, -1.2682595, 0.7093057, -0.29918373, 0.63563055, -0.06036772, 0.3208298, -0.589963, 0.14198181, -3.2224658, -2.3855748, -0.09581307, 0.048893344, 0.28669518, -0.3039601, 1.4152205, 0.6941943, 0.6222726, -2.1048315, 0.6135207, 0.5182585, -0.2081882, -1.1595376, 2.1499195, -0.27442935, 2.0424175, 0.5753505, 0.48386306, 1.1829793, -0.55986434, -0.38761005, -0.33755443, -1.9593081, -0.24775334, -1.2644802, 0.6623538, -0.036791597, 1.006505, -0.085633144, -0.43869615, -0.6538108, -0.5544362, -0.36066625, 2.7083282, -1.7353743, 0.90482634, -0.25781548, 0.4276973, -0.71420306, 1.3187194, 1.6625651, 0.6964582, -0.16635826, -0.916063, 1.4259973, -0.43161312, -3.5115724, -0.96558845, 0.32056043, -1.8625072, 0.35679752, 0.74846184, -1.7382926, -0.26523817, 0.20838377, -0.7228183, -1.1810344, -1.1428597, 2.8348258, 0.6319005, 0.056748897, ...]",png,gs://superrad/inputs/mimic-cxr/files/p10/p10001884/s59305618/9b1a8a51-2b8e4a04-1719059d-aa6bc888-7ace612b.dcm,1376
0,"[-0.79

In [ ]:

combined_df['embedding'].iloc[1]
print(type(combined_df['embedding'].iloc[1]))
print(np.shape(combined_df['embedding'].iloc[1]))

<class 'numpy.ndarray'>
(1376,)


In [ ]:
from google.colab import files
#np.set_printoptions(threshold=np.nan)
combined_df.to_pickle('embeddings.pkl')
files.download('embeddings.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
feature_description = {
        'embedding': tf.io.VarLenFeature(tf.float32),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/id': tf.io.FixedLenFeature([], tf.string), 
}
def parse_example(example_proto):
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)
  
    embedding = tf.sparse.to_dense(parsed_example['embedding'])
    image_format = parsed_example['image/format']
    image_id = parsed_example['image/id']
  
    return embedding, image_format, image_id


all_file_names = []

EMBEDDINGS_DIRECTORY = '/content/data/mimic-embeddings-files'
for i, row in Xray_embeddings.iterrows():

  tfrecord_file = f"{EMBEDDINGS_DIRECTORY}/{row['dicom_id']}.tfrecord"

  all_file_names.append(tfrecord_file)

  

combined_df = pd.DataFrame(columns=['embedding', 'image/format', 'image/id'])

for f in all_file_names:
  tfrecord_file = f
  dataset = tf.data.TFRecordDataset(tfrecord_file)
  dataset = dataset.map(parse_example)

  data_list = []

  for embedding, image_format, image_id in dataset:
      data_list.append({
          'embedding': embedding.numpy(),
          'image/format': image_format.numpy().decode('utf-8'),
          'image/id': image_id.numpy().decode('utf-8')
      })

  df = pd.DataFrame(data_list)
  combined_df = pd.concat([combined_df,df], axis = 0)